In [13]:
file1=open('sample_data/inputs/database.txt', 'r')
Lines1=file1.readlines()
nums=[]
for index,line in enumerate(Lines1):
    line=line.strip()
    if index!=0:
      data=line.split(',') 
      x=[]     
      for ind,i in enumerate(data): 
        if ind!=0:
          x.append(int(i))
      nums.append(x)

In [17]:
def serialTransactions():
  vars={
      'x':0,
      'y':0,
      'z':0,
  } 
  for transactionId in range(1,7): #looping through each transaction
    filename='sample_data/inputs/t'+str(transactionId)+'.txt'
    file=open(filename, 'r')
    Lines=file.readlines()

    schedulerFile=open('sample_data/outputs/schedule1.txt','a')


    from copy import copy, deepcopy
    dummyDb = deepcopy(nums)    
    commands=[]

    for ind,line in enumerate(Lines):
      #line in transaction
      c=line.strip()
      operands=c.split(',')
      
      if c.startswith('START',0,5):
        log=operands[0]+' '+operands[1]+'\n'
        schedulerFile.write(log)

      if c.startswith('READ',0,5): #for READ operation
        numsInd=operands[2]
        if operands[2][0]=='A':
          vars[operands[1]]=dummyDb[0][int(numsInd[1:])-1]
        elif operands[2][0]=='B':
          vars[operands[1]]=dummyDb[1][int(numsInd[1:])-1]
        elif operands[2][0]=='C':
          vars[operands[1]]=dummyDb[2][int(numsInd[1:])-1]
        log=operands[0]+' '+operands[2]+' value into '+operands[1]+' i.e, '+str(vars.get(operands[1]))+'\n'
        schedulerFile.write(log)
      

      elif(c.startswith('WRITE',0,5)): #for WRITE operation
        numsInd=operands[2]
        if operands[2][0]=='A':
          dummyDb[0][int(operands[2][1])-1]=vars.get(operands[1])
        elif operands[2][0]=='B':
          dummyDb[1][int(operands[2][1])-1]=vars.get(operands[1])
        elif operands[2][0]=='C':
          dummyDb[2][int(operands[2][1])-1]=vars.get(operands[1])
        log=operands[0]+' '+operands[1]+' value into '+operands[2]+' i.e, '+str(vars.get(operands[1]))+'\n'
        schedulerFile.write(log)

      elif(c.startswith('EXECUTE',0,7)): #for EXECUTE operation
        if '=' in operands[1]:
          vars[operands[1][0]]=vars.get(operands[1][2])+vars.get(operands[1][4])
        else:
          vars[operands[1][0]]=vars.get(operands[1][0])+int(operands[1][2])
        log=operands[0]+' operation '+operands[1]+' i.e, '+str(vars.get(operands[1][0]))+'\n'
        schedulerFile.write(log)

      elif(c.startswith('COMMIT',0,6)): #for COMMIT operation
        log=operands[0]+' transaction T'+str(transactionId)+'\n'
        schedulerFile.write(log)


      elif(c.startswith('END',0,6)): #for END operation
        log=operands[0]+' transaction T'+str(transactionId)+'\n'
        schedulerFile.write(log)
    file.close()
    schedulerFile.close()

    f = open("sample_data/outputs/database1.txt", "w")
    f.write(Lines1[0])
    for ind,i in enumerate(dummyDb):
      ln=''+str((ind+1))+','
      f.write(ln)
      for j in i:
        ln=''+str(j)+','
        f.write(ln)
      f.write('\n')
    f.close()



serialTransactions()

In [18]:
def interleavingTransactions():
  vars={
      'x':0,
      'y':0,
      'z':0,
  }
  from copy import copy, deepcopy
  dummyDb = deepcopy(nums) 
  schedulerFile=open('sample_data/outputs/schedule2.txt','a')

  def executeCommand(command):
    transId=0
    operands=command.split(',')
    if command.startswith('START',0,5):
      transId=int(operands[1][1])
      log=operands[0]+' '+operands[1]+'\n'
      schedulerFile.write(log)
    
    elif command.startswith('END',0,6):
      log=operands[0]+' transaction T'+str(transId)+'\n'
      schedulerFile.write(log)

    if command.startswith('READ',0,5):
      #READ
      numsInd=operands[2]
      if operands[2][0]=='A':
        vars[operands[1]]=dummyDb[0][int(numsInd[1:])-1]
      elif operands[2][0]=='B':
        vars[operands[1]]=dummyDb[1][int(numsInd[1:])-1]
      elif operands[2][0]=='C':
        vars[operands[1]]=dummyDb[2][int(numsInd[1:])-1]
      log=operands[0]+' '+operands[2]+' value into '+operands[1]+' i.e, '+str(vars.get(operands[1]))+'\n'
      schedulerFile.write(log)


    elif command.startswith('WRITE',0,5): 
      #WRITE  
      numsInd=operands[2]
      if operands[2][0]=='A':
        dummyDb[0][int(operands[2][1])-1]=vars.get(operands[1])
      elif operands[2][0]=='B':
        dummyDb[1][int(operands[2][1])-1]=vars.get(operands[1])
      elif operands[2][0]=='C':
        dummyDb[2][int(operands[2][1])-1]=vars.get(operands[1])
      log=operands[0]+' '+operands[1]+' value into '+operands[2]+' i.e, '+str(vars.get(operands[1]))+'\n'
      schedulerFile.write(log)

    elif command.startswith('EXECUTE',0,7):
      #EXECUTE
      if '=' in operands[1]:
        vars[operands[1][0]]=vars.get(operands[1][2])+vars.get(operands[1][4])
      else:
        vars[operands[1][0]]=vars.get(operands[1][0])+int(operands[1][2])
      log=operands[0]+' operation '+operands[1]+' i.e, '+str(vars.get(operands[1][0]))+'\n'
      schedulerFile.write(log) 


  tQueue=[]
  for transId in range(1,7):# each transaction
    tQueue.append('sample_data/inputs/t'+str(transId)+'.txt')
  pipeline1=[]
  pipeline2=[]

  while len(tQueue)!=0:
    if len(pipeline1)==0:
      f=open(tQueue[0])
      Lines=f.readlines()
      for l in Lines:
        pipeline1.append(l.strip())
      tQueue.pop(0)
    if len(pipeline2)==0:
      f=open(tQueue[0])
      Lines=f.readlines()
      for l in Lines:
        pipeline2.append(l.strip())
      tQueue.pop(0)
    while len(pipeline1)!=0 and len(pipeline2)!=0:
      executeCommand(pipeline1[0])
      pipeline1.pop(0)
      executeCommand(pipeline2[0])
      pipeline2.pop(0)
  f1 = open("sample_data/outputs/database2.txt", "w")
  f1.write(Lines1[0])
  for ind,i in enumerate(dummyDb):
    ln=''+str((ind+1))+','
    f1.write(ln)
    for j in i:
      ln=''+str(j)+','
      f1.write(ln)
    f1.write('\n')
  f1.close()
    
    
interleavingTransactions()

In [19]:
def strictTwoPhaseLocking():
  vars={
      'x':0,
      'y':0,
      'z':0,
  }

  locks = dict()

  from copy import copy, deepcopy
  dummyDb = deepcopy(nums) 
  schedulerFile=open('sample_data/outputs/schedule3.txt','a')
  transId=0

  def executeCommand(command, transId):
    operands=command.split(',')

    if len(operands) > 2 and operands[2] not in locks:
        l = dict()
        l['read'] = 0  
        l['write'] = 0
        locks[operands[2]] = l
    
    if command.startswith('START',0,5):
      log=operands[0]+' '+operands[1]+'\n'
      schedulerFile.write(log)
    
    elif command.startswith('END',0,6):
      log=str(transId)+' '+operands[0]+' transaction T'+str(transId)+'\n'
      schedulerFile.write(log)

    elif command.startswith('COMMIT', 0, 6):
      log=operands[0]+' transaction T'+str(transId)+'\n'
      schedulerFile.write(log)
      for k, v in locks.items():
        if locks[k]['write'] == transId:
          locks[k]['write'] = 0;

    if command.startswith('READ',0,5):
      #READ

      if locks[operands[2]]['write'] !=0 and locks[operands[2]]['write'] != transId:
        return False
      locks[operands[2]]['read'] = transId   
      
      numsInd=operands[2]
      if operands[2][0]=='A':
        vars[operands[1]]=dummyDb[0][int(numsInd[1:])-1]
      elif operands[2][0]=='B':
        vars[operands[1]]=dummyDb[1][int(numsInd[1:])-1]
      elif operands[2][0]=='C':
        vars[operands[1]]=dummyDb[2][int(numsInd[1:])-1]
    
      log=str(transId)+' '+operands[0]+' '+operands[2]+' value into '+operands[1]+' i.e, '+str(vars.get(operands[1]))+'\n'
      schedulerFile.write(log)

      locks[operands[2]]['read'] = 0

    elif command.startswith('WRITE',0,5): 
      #WRITE  
      if locks[operands[2]]['write'] != 0 and locks[operands[2]]['write'] != transId:
        return False

      locks[operands[2]]['write'] = transId

      numsInd=operands[2]
      if operands[2][0]=='A':
        dummyDb[0][int(operands[2][1])-1]=vars.get(operands[1])
      elif operands[2][0]=='B':
        dummyDb[1][int(operands[2][1])-1]=vars.get(operands[1])
      elif operands[2][0]=='C':
        dummyDb[2][int(operands[2][1])-1]=vars.get(operands[1])
      log=str(transId)+' '+operands[0]+' '+operands[1]+' value into '+operands[2]+' i.e, '+str(vars.get(operands[1]))+'\n'
      schedulerFile.write(log)


    elif command.startswith('EXECUTE',0,7):
      #EXECUTE
      if '=' in operands[1]:
        vars[operands[1][0]]=vars.get(operands[1][2])+vars.get(operands[1][4])
      else:
        vars[operands[1][0]]=vars.get(operands[1][0])+int(operands[1][2])
      log=str(transId)+' '+operands[0]+' operation '+operands[1]+' i.e, '+str(vars.get(operands[1][0]))+'\n'
      schedulerFile.write(log)

    return True


  tQueue=[]
  for tId in range(1,7):# each transaction
    tQueue.append('sample_data/inputs/t'+str(tId)+'.txt')
  pipeline1=[]
  pipeline2=[]

  p1Id = 0
  p2Id = 0

  while len(tQueue)!=0:
    if len(pipeline1)==0:
      f=open(tQueue[0])
      Lines=f.readlines()
      for l in Lines:
        pipeline1.append(l.strip())
      p1Id = 7 - len(tQueue)
      tQueue.pop(0)
    if len(pipeline2)==0:
      f=open(tQueue[0])
      Lines=f.readlines()
      for l in Lines:
        pipeline2.append(l.strip())
      p2Id = 7 - len(tQueue)
      tQueue.pop(0)

    breaker = 0;
    
    while len(pipeline1)!=0 and len(pipeline2)!=0:
      breaker+= 1
      if executeCommand(pipeline1[0], p1Id):
        pipeline1.pop(0)
        breaker = 0
      if executeCommand(pipeline2[0], p2Id):
        pipeline2.pop(0)

      if breaker > 1000:
        break
    if breaker > 1000:
      schedulerFile.write("Deadlocked!! with transactions "+str(p1Id)+" and "+str(p2Id))
      break
  f1 = open("sample_data/outputs/database3.txt", "w")
  f1.write(Lines1[0])
  for ind,i in enumerate(dummyDb):
    ln=''+str((ind+1))+','
    f1.write(ln)
    for j in i:
      ln=''+str(j)+','
      f1.write(ln)
    f1.write('\n')
  f1.close()


strictTwoPhaseLocking()